<a href="https://colab.research.google.com/github/fabriciocarraro/CrewAI_agent-Tech_job_tailored_CV_creator/blob/main/CrewAI_agent_Tech_job_tailored_CV_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U  crewai crewai_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 657.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.7/462.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew

**Note**:
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

os.environ["SERPER_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool("/content/fake_cv.md")
semantic_search_resume = MDXSearchTool(mdx="/content/fake_cv.md")

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.88it/s]


## Creating Agents

In [ ]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in navigating and extracting critical information from job postings is unmatched."
        "Your skills help pinpoint the necessary qualifications and skills sought by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [ ]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants to help them stand out in the job market",
    tools = [scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect and synthesize information from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the groundwork for personalized resume enhancements."
    )
)

In [ ]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they resonate perfectly with the job's requirements."
    )
)

In [ ]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [ ]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [ ]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, project experiences, contributions, interests, and communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [ ]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, including the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [ ]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [ ]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

In [ ]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership roles that require a strategic and innovative approach."""
}

In [ ]:
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [2024-08-29 02:04:33][DEBUG]: == Working Agent: Tech Job Researcher
 [2024-08-29 02:04:33][INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [2024-08-29 02:04:33][DEBUG]: == Working Agent: Personal Profiler for Engineers
 [2024-08-29 02:04:33][INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led major tech initiatives and startu

- Display the generated `tailored_resume.md` file.

In [ ]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Noah Williams

*Email: noah.williams@example.dev*  
*Phone: +44 11 111 11111*

## Profile
Innovative Full Stack Engineer with over 18 years of experience in software development, specializing in both front-end and back-end technologies. Strong expertise in building scalable applications and managing the full software development life cycle from conception to deployment. Proficient in a wide range of programming languages and frameworks, including Ruby, Python, JavaScript, TypeScript, Elixir, React, and Angular. Proven ability to collaborate effectively with cross-functional teams, ensuring that technical solutions align with business goals. Passionate about leveraging AI and machine learning to create impactful software solutions.

## Skills
- **Programming Languages:** Ruby, Python, JavaScript, TypeScript, Elixir
- **Frameworks:** Ruby on Rails, React, Angular, Node.js, Phoenix
- **Database Management:** MySQL, MongoDB
- **Web Technologies:** HTML, CSS, JavaScript, XML, jQuery
- **API Development:** Experience writing effective APIs and managing server-side integration
- **Project Management:** Agile methodologies, Cross-functional Collaboration, Strategic Planning
- **Data Science & AI:** Strong understanding of AI/ML models and data handling techniques
- **UI/UX Design:** Familiarity with creating appealing visual designs and user experiences

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Transformed the engineering division into a critical revenue driver by expanding the customer base through innovative software solutions.
- Integrated cutting-edge AI technologies and scalable databases, enhancing product functionalities and market positioning.
- Led a team focused on delivering high-quality software solutions, ensuring responsiveness and efficiency through continuous testing and debugging.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Directed engineering strategy in collaboration with C-level executives, shaping the company's technological direction.
- Managed cross-functional teams to develop applications for both desktop and mobile platforms, ensuring seamless user experiences.
- Fostered a culture of mentorship and professional growth, enhancing team capabilities and project outcomes.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Founded and led a startup focused on IoT solutions, overseeing product development from concept to execution, including the development of a GPS tracking collar.
- Successfully built the initial product using MongoDB and managed the deployment of features and updates.
- Established production facilities and navigated early-stage funding, showcasing entrepreneurial skills and innovative thinking.

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Managed multiple engineering teams, driving productivity and innovation while implementing Ruby on Rails and React for new product development.
- Enhanced inter-departmental coordination, leading to improved project outcomes and team morale.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Led the development of a high-performance engineering team, achieving a 500% increase in company revenue within two years.
- Integrated data analytics into business strategies, developing predictive modeling tools for enhanced customer insights.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed a central API that improved functionality for a large engineering team and thousands of users, significantly enhancing system performance.
- Implemented caching strategies to optimize response times and reduce system loads.

### BetCraft: CTO — 2013 - 2015
- Led technology strategy post-Series A funding, guiding significant advancements in platform performance and market reach.
- Built initial products using React and Angular, gaining expertise in full stack development.

## Education
- **MBA in Information Technology**  
  London Business School - MBA
- **Advanced Leadership Techniques**  
  University of London - Certification
- **Data Science Specialization**  
  Coursera (Johns Hopkins University) - Certification
- **B.Sc. in Computer Science**  
  University of Edinburgh - Bachelor’s degree

## Interests
- **Artificial Intelligence & Machine Learning:** Committed to leveraging AI to solve complex problems and improve software solutions.
- **Open Source Contributions:** Active participant in open-source projects, contributing to community-driven development.
- **Mentorship:** Dedicated to mentoring emerging tech leaders and fostering a culture of collaboration and learning.

---

This updated resume effectively highlights Noah William's qualifications and experiences relevant to the Full Stack Engineer position, ensuring it resonates with the job requirements at AI Fund.

- Display the generated `interview_materials.md` file.

In [ ]:
display(Markdown("./interview_materials.md"))

### Interview Questions and Talking Points for Noah Williams

#### General Questions

1. **Can you describe your experience with the full software development life cycle?**
   - **Talking Point:** Highlight your 18 years of experience in software development, emphasizing your roles in managing projects from conception through deployment at DataKernel and InnovPet.

2. **What programming languages and frameworks are you most comfortable with?**
   - **Talking Point:** Discuss your proficiency in Ruby, Python, JavaScript, TypeScript, and Elixir, as well as your experience with frameworks like Ruby on Rails, React, and Angular.

3. **How do you approach collaboration with product managers and cross-functional teams?**
   - **Talking Point:** Share examples from your experience at DataKernel, where you directed engineering strategies in collaboration with C-level executives, and how that fostered effective teamwork.

#### Technical Skills

4. **Can you explain your experience in building APIs? What challenges have you faced?**
   - **Talking Point:** Discuss your experience at DriveAI developing a central API and the optimizations you implemented, such as advanced caching strategies that improved system performance.

5. **What strategies do you implement to ensure the quality and efficiency of the software you build?**
   - **Talking Point:** Reference your approach to testing and debugging at DataKernel, where you ensured responsiveness and efficiency through continuous testing practices.

6. **How do you ensure that your applications are secure and protect user data?**
   - **Talking Point:** Discuss any relevant experiences related to creating security settings and data protection measures in your past projects.

#### Project Management and Leadership

7. **Can you provide an example of a project where you had to troubleshoot and debug significant issues?**
   - **Talking Point:** Share a specific instance from your work history, perhaps at PrintPack, where your leadership led to resolving critical issues and enhancing project outcomes.

8. **How do you prioritize tasks and manage multiple projects simultaneously?**
   - **Talking Point:** Talk about your project management skills developed through Agile methodologies and your experience overseeing multiple teams at EliteDevs.

9. **What is your approach to mentoring and developing your team members?**
   - **Talking Point:** Highlight your initiatives at DataKernel to foster a culture of mentorship and professional growth, ensuring team members achieve their potential.

#### Understanding Business Needs

10. **How do you align technical solutions with business goals?**
    - **Talking Point:** Discuss your experience in shaping DataKernel’s technological trajectory in alignment with business objectives and your role in ensuring that software solutions meet market needs.

11. **Can you describe a situation where you had to adapt to a changing business environment?**
    - **Talking Point:** Reference your entrepreneurial experience at InnovPet, where you navigated changing market conditions and successfully led product development.

#### Additional Considerations

12. **What interests you about working for AI Fund, and how do you see yourself contributing to our mission?**
    - **Talking Point:** Express your passion for AI and machine learning and how your experience aligns with AI Fund's commitment to innovation in technology.

13. **How do you stay updated with the latest technologies and industry trends?**
    - **Talking Point:** Mention your engagement in open-source projects and continuous learning through certifications, such as your Data Science Specialization.

14. **What is your understanding of diversity and inclusion in the workplace? Why is it important?**
    - **Talking Point:** Share your views on fostering an inclusive environment based on mutual respect and how it contributes to company success, referencing AI Fund's commitment to diversity.

### Conclusion
Prepare to convey your extensive experience, technical skills, and leadership qualities while aligning them with the requirements of the Full Stack Engineer position at AI Fund. Practice your responses to these questions to build confidence and ensure you effectively communicate your fit for the role.